In [1]:
# all imported libraries
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import cv2

In [2]:
train_data = datasets.MNIST(
    root="data",
    train=True,  # load train set
    transform=ToTensor(),
    download=True  # download the dataset
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True  # download the dataset
)

In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [6]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [7]:
# load the train and test data in batches, shuffle it, etc
loaders = {
    "train" : DataLoader(train_data, batch_size=100,shuffle=True, num_workers=1),
    "test" : DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1)  
}

loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1cf8f2977d0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x1cf8d831f10>}

In [8]:
# define the model architecture
# the neural network structure

class CNN(nn.Module):  # inherits from the nn.Module
    def __init__(self):
        # call the super class
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # takes 1 channel in and 10 out
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  # takes 10 channels in and 20 out
        self.conv2_dropout = nn.Dropout2d()  # dropout layer is a regularization layer, it will randomly (based on a probability) deactivate certain network nodes
        self.fcl = nn.Linear(320, 50)  # takes 320 in and 50 out
        self.fcl2 = nn.Linear(50, 10)  # takes 50 in and 10 out (IT HAS TO TAKE 10 OUT (0-9) = 10 digits)

    def forward(self, x):  # this will define the activations
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_dropout(self.conv2(x)), 2))

        # reshape/flatten the data
        x = x.view(-1, 320)

        x = F.relu(self.fcl(x))
        x = F.dropout(x, training=self.training)
        x = self.fcl2(x)

        return F.softmax(x)

In [9]:
# optimize and train the data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# define a loss functin
loss_fn = nn.CrossEntropyLoss()

# define the training process
def train(epoch):
    model.train()

    for batch_i, (data, target) in enumerate(loaders["train"]):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        optimizer.zero_grad()


        if batch_i % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_i * len(data)}/{len(loaders['train'].dataset)} ({100. * batch_i / len(loaders['train'].dataset):.0f}%)]\t{loss.item():.6f}")


def test():
    model.eval()

    test_loss = 0
    correct = 0 

    with torch.no_grad():
        for data, target in loaders["test"]:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders["test"].dataset)
    print(f"\nTest Set: Average Loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders['test'].dataset)} ({100. * correct / len(loaders['test'].dataset):.0f}%\n)")         

In [10]:
# call functions
for epoch in range(1, 11):
    train(epoch)
    test()

C:\Users\bride\AppData\Local\Temp\ipykernel_19448\1752899068.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.302736
Train Epoch: 1 [2000/60000 (0%)]	2.296676
Train Epoch: 1 [4000/60000 (0%)]	2.197555
Train Epoch: 1 [6000/60000 (0%)]	1.931864
Train Epoch: 1 [8000/60000 (0%)]	1.783849
Train Epoch: 1 [10000/60000 (0%)]	1.791260
Train Epoch: 1 [12000/60000 (0%)]	1.857669
Train Epoch: 1 [14000/60000 (0%)]	1.672479
Train Epoch: 1 [16000/60000 (0%)]	1.702713
Train Epoch: 1 [18000/60000 (0%)]	1.762462
Train Epoch: 1 [20000/60000 (0%)]	1.730056
Train Epoch: 1 [22000/60000 (0%)]	1.671901
Train Epoch: 1 [24000/60000 (0%)]	1.683005
Train Epoch: 1 [26000/60000 (0%)]	1.707954
Train Epoch: 1 [28000/60000 (0%)]	1.713817
Train Epoch: 1 [30000/60000 (0%)]	1.648243
Train Epoch: 1 [32000/60000 (1%)]	1.668981
Train Epoch: 1 [34000/60000 (1%)]	1.686297
Train Epoch: 1 [36000/60000 (1%)]	1.671487
Train Epoch: 1 [38000/60000 (1%)]	1.598917
Train Epoch: 1 [40000/60000 (1%)]	1.672168
Train Epoch: 1 [42000/60000 (1%)]	1.601570
Train Epoch: 1 [44000/60000 (1%)]	1.645505
Train Epoch: 1 [460

In [14]:
# save the model
torch.save(model, "./model/handwritten_CNN_model.pth")

In [ ]:
data, labels = test_data[0]

data.shape

In [ ]:
"""
WHAT IM THINKING: get the image, turn it into a tensor then feed it into the model
and print out the CNNs prediction
"""
# get the image and turn it into gray scale
img_path = "handwriting/image0.png"  # image path here
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28,28))

print(img.shape)  # (530, 800)  -> (28, 28)

# turn to a tensor
data = torch.from_numpy(img).unsqueeze(0).float() / 255.0
# print(data.min().item(), data.max().item())

data = torch.reshape(data, (1, 28, 28))
# print(data)
# print(data.shape)


output = model(data)
prediction = output.argmax(dim=1, keepdim=True).item()

plt.imshow(img, cmap="gray")
print(f"Prediction: {prediction}")

